In [10]:
# Code to read csv file into colaboratory:
!pip install sentence-transformers

     |████████████████████████████████| 79 kB 3.7 MB/s 
     |████████████████████████████████| 4.0 MB 11.3 MB/s 
     |████████████████████████████████| 1.2 MB 44.9 MB/s 
     |████████████████████████████████| 77 kB 6.5 MB/s 
     |████████████████████████████████| 596 kB 51.4 MB/s 
     |████████████████████████████████| 6.5 MB 30.9 MB/s 
     |████████████████████████████████| 895 kB 50.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=b07c43d8fd3e23a465cefdad5b2d6e3fb893c67fe0966c669502f2699bfb9018
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import numpy as np
import pandas as pd
import nltk
# nltk.download('punkt')
import sentence_transformers
from sentence_transformers import SentenceTransformer, models
data_set=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/global-headline-only.csv')[['event_type', 'cleaned']]
data_set.head(1)

,event_type,cleaned
0,Explosions/Remote violence,regime forces shelled Arnaba town in Idleb cou...


In [13]:
print (f"Shape of original data set is {data_set.shape}")

Shape of original data set is (1034527, 2)


In [14]:
df=data_set.sample(100000).reset_index().drop(['index'], axis = 1)
df.head(5)

,event_type,cleaned
0,Protests,nanbai associations protested in Abbottabad ci...
1,Battles,military forces of Azerbaijan fired with small...
2,Riots,in Santiago at least rioters clashed with Pol...
3,Violence against civilians,an unidentified armed group shot and injured t...
4,Protests,an unreported umber of people staged a Basic ...


In [15]:
del data_set

In [16]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

In [18]:
train = model.encode(df.cleaned, device='cuda')

In [17]:
from sklearn import preprocessing
# features_trainl encode the target variable

encoder = preprocessing.LabelEncoder()
df["event_type_num"] = encoder.fit_transform(df["event_type"])

In [19]:
df_features = pd.DataFrame()
df_features['cleaned_emb'] = [i for i in train] 
df_features['event_type'] = df["event_type_num"]
df_features.head(3)

,cleaned_emb,event_type
0,"[0.15117389, 0.34569043, -0.0374072, 0.0231863...",2
1,"[0.11691147, 0.364508, 0.057245333, 0.51362205...",0
2,"[-0.00610396, 0.2031562, 0.066696495, 0.189912...",3


In [20]:
# df_features.to_csv("./features_set.csv")

In [21]:
from sklearn.decomposition import PCA
import time
pca_model = PCA(n_components=50)
start = time.process_time()
pca_model.fit(train)
print("Sum of variance ratios: ",sum(pca_model.explained_variance_ratio_))
print(time.process_time() - start)

Sum of variance ratios:  0.5963824776103331
12.07283436800003


In [22]:
x_comps = pca_model.transform(train)
x_comps.shape

(100000, 50)

In [23]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_comps,df_features['event_type'],test_size=0.2,random_state=42)

In [24]:
from sklearn.svm import SVC
start = time.time() 

svm_classifier = SVC()
svm_classifier.fit(x_train,y_train)

end = time.time()
process = round(end-start,2)
print("Support Vector Machine Classifier has fitted, this process took {} seconds".format(process))

Support Vector Machine Classifier has fitted, this process took 93.36 seconds


In [25]:
svm_classifier.score(x_test,y_test)

0.92725

In [26]:
# More algorithms!!!!
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB,BernoulliNB

rfc = RandomForestClassifier()
rfc.fit(x_train,y_train)
print("Score of RFC",rfc.score(x_test,y_test))

logreg = LogisticRegression()
logreg.fit(x_train,y_train)
print("Score of LogReg",logreg.score(x_test,y_test))

gnb = GaussianNB()
gnb.fit(x_train,y_train)
print("Score of GaussianNB",gnb.score(x_test,y_test))

bnb = BernoulliNB()
bnb.fit(x_train,y_train)
print("Score of BernoulliNB",bnb.score(x_test,y_test))

Score of RFC 0.89775
Score of LogReg 0.85885
Score of GaussianNB 0.82905
Score of BernoulliNB 0.76595
